In [17]:
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# Spot for user-specified options
# This script is built to access only the geomet daily climate data
# Site for discovery of climate obs and IDs: https://api.weather.gc.ca/collections/climate-daily/items?limit=10&offset=0&bbox=-180%2C34.84%2C131.89%2C82.41&sortby=-LOCAL_DATE
# Queryable parameters: https://api.weather.gc.ca/collections/climate-daily/queryables# 

# ------ User-specified options below ------
# dt_start/end: String, format=YYYY-MM-DD, enclosed in quotes
# clim_ids: Strings ECCC Climate Identifiers, separated by pipe (|) for multiple. Enclosed in quotes
# params: List of available parameters to return, separated by '%2C'. LOCAL_DATE is required. 
# api_limit: Integer. The limit per API request. Max is 10000 for Geomet daily climate data.
dt_start = "1990-01-01"
dt_end = "2025-12-21"
clim_ids = "2101300|2101303|2101415"
params = "ID%2CLOCAL_DATE%2CMAX_TEMPERATURE%2CMIN_TEMPERATURE%2CMEAN_TEMPERATURE" 
api_limit = 10000
# ---- End of user-specified options. Nothing below here should be changed. 

# Options: From geomet doc page. Put all options here
base_url = "https://api.weather.gc.ca/collections/climate-daily/items"
url_opts = "".join([f"?f=csv&limit={api_limit}",
                    f"&properties={params}",
                    "&sortby=LOCAL_DATE"])
clim_id = "".join([f"&CLIMATE_IDENTIFIER={clim_ids}"])

# Calculate number of days requested
start = datetime.strptime(dt_start, '%Y-%m-%d')
end = datetime.strptime(dt_end, '%Y-%m-%d')
total_days = (end - start).days + 1

geo_urls = []
if total_days <= api_limit:
    geo_url = "".join([base_url, url_opts, f"&datetime={dt_start}/{dt_end}", clim_id])
    geo_urls = [geo_url]
else:
    # First call: up to api_limit days
    end1 = start + timedelta(days=api_limit - 1)
    dt_string1 = f"&datetime={dt_start}/{end1.strftime('%Y-%m-%d')}"
    geo_url1 = "".join([base_url, url_opts, dt_string1, clim_id])
    geo_urls.append(geo_url1)
    
    # Second call: remainder
    start2 = end1 + timedelta(days=1)
    dt_string2 = f"&datetime={start2.strftime('%Y-%m-%d')}/{dt_end}"
    geo_url2 = "".join([base_url, url_opts, dt_string2, clim_id])
    geo_urls.append(geo_url2)

# For backward compatibility, set geo_url to the first one if single
if len(geo_urls) == 1:
    geo_url = geo_urls[0]


In [19]:
# Retrieve data into a pandas df
if 'geo_urls' in locals() and len(geo_urls) > 1:
    df = pd.concat([pd.read_csv(url) for url in geo_urls], ignore_index=True)
else:
    df = pd.read_csv(geo_url)

In [ ]:
# Convert data types
df['LOCAL_DATE'] = pd.to_datetime(df['LOCAL_DATE'])
df['ID'] = df['ID'].astype(str)

# Replace any non-numeric characters in temperature columns with NA for the purposes of sorting. 
params_list = params.split('%2C')
variable_cols = params_list[params_list.index('LOCAL_DATE') + 1:]
df[variable_cols] = df[variable_cols].apply(pd.to_numeric, errors='coerce')


# Sort and drop duplicates.
df = df.sort_values(['LOCAL_DATE','MIN_TEMPERATURE','MAX_TEMPERATURE']).drop_duplicates('LOCAL_DATE')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12973 entries, 0 to 13122
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   x                 12973 non-null  float64       
 1   y                 12973 non-null  float64       
 2   ID                12973 non-null  object        
 3   LOCAL_DATE        12973 non-null  datetime64[ns]
 4   MAX_TEMPERATURE   12885 non-null  float64       
 5   MIN_TEMPERATURE   12903 non-null  float64       
 6   MEAN_TEMPERATURE  12880 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 810.8+ KB


In [21]:
# Check for missing dates
min_date = df['LOCAL_DATE'].dt.date.min()
max_date = df['LOCAL_DATE'].dt.date.max()
all_dates = pd.date_range(start=min_date, end=max_date, freq='D')
existing_dates = df['LOCAL_DATE'].dt.date
missing_dates = all_dates.difference(existing_dates)



In [22]:
# Find periods where MAX_TEMPERATURE did not exceed -20.0 for 3 or more consecutive days
# First, create a dataframe with all dates filled
min_date = df['LOCAL_DATE'].min().date()
max_date = df['LOCAL_DATE'].max().date()
full_dates = pd.date_range(start=min_date, end=max_date, freq='D')
df_full = df.set_index('LOCAL_DATE').reindex(full_dates).rename_axis('LOCAL_DATE').reset_index()

# Condition: MAX_TEMPERATURE <= -20.0 or NaN (treat NaN as part of streak)
condition = (df_full['MAX_TEMPERATURE'] <= -20.0) | df_full['MAX_TEMPERATURE'].isna()

# Identify consecutive streaks
df_full['streaknum'] = (condition != condition.shift()).cumsum()

# Group by streak and get periods where condition is True for >=3 days
streaks = df_full[condition].groupby('streaknum').agg(
    start=('LOCAL_DATE', 'min'),
    end=('LOCAL_DATE', 'max'),
    count=('LOCAL_DATE', 'count'),
    missing=('MAX_TEMPERATURE', lambda x: x.isna().sum())
)
long_streaks_max = streaks[streaks['count'] >= 5].sort_values('count', ascending=False)
long_streaks_max['type'] = 'MAX_TEMPERATURE'

print("Periods where MAX_TEMPERATURE did not exceed -20.0 for 5 or more consecutive days:")
print(long_streaks_max)

Periods where MAX_TEMPERATURE did not exceed -20.0 for 5 or more consecutive days:
               start        end  count  missing             type
streaknum                                                       
163       1996-01-08 1996-01-30     23       19  MAX_TEMPERATURE
271       2007-04-08 2007-04-30     23       23  MAX_TEMPERATURE
3         1990-01-26 1990-02-12     18        0  MAX_TEMPERATURE
287       2008-12-28 2009-01-12     16        0  MAX_TEMPERATURE
587       2025-12-08 2025-12-21     14        0  MAX_TEMPERATURE
279       2008-01-27 2008-02-09     14        0  MAX_TEMPERATURE
161       1995-11-27 1995-12-09     13        0  MAX_TEMPERATURE
335       2012-12-16 2012-12-27     12        6  MAX_TEMPERATURE
333       2012-11-27 2012-12-08     12        5  MAX_TEMPERATURE
17        1990-12-31 1991-01-09     10        0  MAX_TEMPERATURE
185       1998-01-01 1998-01-10     10        0  MAX_TEMPERATURE
473       2020-01-10 2020-01-18      9        0  MAX_TEMPERATURE
545    

In [23]:
# Find periods where MIN_TEMPERATURE reached at least -30.0 for 3 or more consecutive days
# First, create a dataframe with all dates filled
min_date = df['LOCAL_DATE'].min().date()
max_date = df['LOCAL_DATE'].max().date()
full_dates = pd.date_range(start=min_date, end=max_date, freq='D')
df_full = df.set_index('LOCAL_DATE').reindex(full_dates).rename_axis('LOCAL_DATE').reset_index()

# Condition: MIN_TEMPERATURE <= -30.0 or NaN (treat NaN as part of streak)
condition = (df_full['MIN_TEMPERATURE'] <= -30.0) | df_full['MIN_TEMPERATURE'].isna()

# Identify consecutive streaks
df_full['streak'] = (condition != condition.shift()).cumsum()

# Group by streak and get periods where condition is True for >=3 days
streaks = df_full[condition].groupby('streak').agg(
    start=('LOCAL_DATE', 'min'),
    end=('LOCAL_DATE', 'max'),
    count=('LOCAL_DATE', 'count'),
    missing=('MIN_TEMPERATURE', lambda x: x.isna().sum())
)
long_streaks_min = streaks[streaks['count'] >= 5].sort_values('count', ascending=False)
long_streaks_min['type'] = 'MIN_TEMPERATURE'

print("Periods where MIN_TEMPERATURE reached at least -30.0 for 5 or more consecutive days:")
print(long_streaks_min)

Periods where MIN_TEMPERATURE reached at least -30.0 for 5 or more consecutive days:
            start        end  count  missing             type
streak                                                       
309    2007-04-08 2007-04-30     23       23  MIN_TEMPERATURE
177    1996-01-12 1996-01-29     18        0  MIN_TEMPERATURE
325    2008-12-30 2009-01-10     12        0  MIN_TEMPERATURE
197    1998-01-01 1998-01-10     10        0  MIN_TEMPERATURE
531    2020-01-10 2020-01-19     10        0  MIN_TEMPERATURE
477    2015-02-01 2015-02-10     10        0  MIN_TEMPERATURE
5      1990-01-26 1990-02-03      9        0  MIN_TEMPERATURE
19     1990-12-31 1991-01-08      9        0  MIN_TEMPERATURE
317    2008-01-27 2008-02-04      9        0  MIN_TEMPERATURE
545    2021-02-08 2021-02-16      9        0  MIN_TEMPERATURE
267    2004-01-24 2004-01-31      8        0  MIN_TEMPERATURE
173    1995-12-03 1995-12-10      8        0  MIN_TEMPERATURE
359    2012-01-15 2012-01-22      8        0  M

In [24]:
# Export the stats and the list of missing data to csv files in the current wd

# List of missing dates
df_missing = pd.DataFrame({'missing_dates': missing_dates})
df_missing.to_csv('missing_dates.csv', index=False)
print("Missing dates exported to missing_dates.csv")

# df of all sorted data
df.to_csv('geomet_climate_data.csv', index=False)
print("Geomet climate data exported to geomet_climate_data.csv")

# streak dataframes combined
all_streaks = pd.concat([long_streaks_max, long_streaks_min], ignore_index=True)
all_streaks.to_csv('cold_streaks.csv', index=False)
print("Cold streaks data exported to cold_streaks.csv")

Missing dates exported to missing_dates.csv
Geomet climate data exported to geomet_climate_data.csv
Cold streaks data exported to cold_streaks.csv
